In [1]:
from spacy.tokens import DocBin
import spacy
import json
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
import random

In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [4]:
camp = load_data("data/camp.json")
print (camp[0])

{'text': 'Retrieved from "https://en.wikipedia.org/wiki/Matthew_Justus"Not to be confused with Columbushaus.  KZ Columbia Memorial, diagonally opposite to its former site now covered by the airport building Columbia concentration camp  was a Nazi concentration camp situated in the Tempelhof area of Berlin. It was one of the first such institutions established by the regime.', 'spans': [{'start': 290, 'end': 296, 'label': 'CAMP'}]}


In [5]:
def convert_data(data):
    final = []
    for item in data:
        text = item["text"]
        ents = item["spans"]
        final_ents = []
        for ent in ents:
            start, end, label = ent["start"], ent["end"], ent["label"]
            final_ents.append((start, end, label))
        ents = {"entities": final_ents}
        final.append([text, ents])
    return (final)

In [6]:
camp = convert_data(camp)
random.shuffle(camp)
camp_train = camp[0:500]
camp_valid = camp[501:1000]
# write_data("data/camp_train.json", camp_train)
# write_data("data/camp_valid.json", camp_valid)
write_data("data/camp_train_GG.json", camp_train)
write_data("data/camp_valid_GG.json", camp_valid)
print (camp_train[0])

['The Greek campaign ended with a complete German and Italian victory. The British did not have the military resources to carry out big simultaneous operations in North Africa and the Balkans. Even if they had been able to block the Axis advance, they would have been unable to exploit the situation by a counter-thrust across the Balkans. The British came very near to holding Crete and perhaps other islands that would have provided air support for naval operations throughout the eastern Mediterranean. In enumerating the reasons for the complete Axis victory in Greece, the following factors were of greatest significance:', {'entities': [(41, 47, 'CAMP')]}]


In [7]:
nlp = spacy.blank("en") 
def create_training(TRAIN_DATA):
    db = DocBin() # create a DocBin object
    for text, annot in tqdm(TRAIN_DATA): # data in previous format
        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        for start, end, label in annot["entities"]: # add character indexes
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents # label the text with the ents
        db.add(doc)
    return (db)

In [8]:
camp_train = create_training(camp_train)
# camp_train.to_disk("./data/camp_train.spacy")
camp_train.to_disk("./data/camp_train_GG.spacy")

camp_valid = create_training(camp_valid)
# camp_valid.to_disk("./data/camp_valid.spacy")
camp_train.to_disk("./data/camp_train_GG.spacy")


100%|██████████| 499/499 [00:01<00:00, 363.93it/s]
